In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Negamax with Alpha-Beta Pruning

### Overview
**Alpha-beta pruning** is an optimization that can be applied to the negamax search algorithm. It was described and implemented in various ways throughout history. According to [Knuth, D. E. and Moore, R. W. (1975)](Bibliography.ipynb#KM75), it was John McCarthy who first thought of the concept of pruning a search tree in the context of a chess algorithm, when he criticized Alex Bernstein for his more naive chess program during the Dartmouth Summer Research Conference on Artificial Intelligence in 1956. After that, the first published discussion of alpha-beta pruning was made by Allen Newell, Herbert Simon and Cliff Shaw. Donald E. Knuth and Ronald W. Moore present a more refined version of alpha-beta pruning in their paper. The algorithm presented in this notebook is based on this refined version. 

Alpha-beta pruning aims to "cut off" (prune) any branches in the search tree that cannot affect the outcome, therefore not evaluating these branches and speeding up the search. The algorithm achieves this by keeping track of two additional parameters during the search, which are denoted as $\alpha$ and $\beta$. $\alpha$ can be considered the lower bound; it is the lowest possible value that the search could output at that time. In contrast, $\beta$ can be considered the upper bound; it is the highest possible value that the search could output at that point in time. $\alpha$ and $\beta$ are regularly updated during the search to narrow down the possible range of values.  

### Finding the Best Value
Based on this idea, we can define a new function $\texttt{bestValueAlphaBeta}$ with the following signature:

$$\texttt{bestValueAlphaBeta: States}\times \texttt{Players} \times \{-1, 1\} \times \mathbb{N} \cup \{0\} \times  \texttt{Paths} \times \mathbb{Z} \times \mathbb{Z} \rightarrow \mathbb{Z}$$

$\texttt{bestValueAlphaBeta}$ approximates the function $\texttt{bestValue}$ as described in [NegamaxAlgorithm.ipynb](NegamaxAlgorithm.ipynb#Negamax-Search), but applies the aforementioned alpha-beta pruning optimization. The parameters of $\texttt{bestValueAlphaBeta}$ are the same as those of $\texttt{bestValue}$, but with the addition of $\alpha$ and $\beta$. We can define $\texttt{bestValueAlphaBeta}$ based on three cases:

1. $\alpha < \texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) < \beta \ \rightarrow\ \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta) = \texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path})$
1. $\texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) \leq \alpha \rightarrow \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta) \leq \alpha$
1. $\beta \leq \texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) \rightarrow \beta \leq \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta)$

As a side note, if the upper bound $\texttt{ub}$ and the lower bound $\texttt{lb}$ of $\texttt{bestValue}$ are known, the following is true:

$$\texttt{bestValue}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}) := \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \texttt{lb}, \texttt{ub})$$

### Our Implementation
Naturally, there are multiple implementations that meet this definition. This notebook presents one such implementation, which utilizes these properties in an attempt to improve performance compared to negamax search without alpha-beta pruning.

Our implementation is given by the function `AlphaBetaPruning.get_best_move`, which has the same parameters as `NegamaxSearch.get_best_move` with the addition of $\alpha$ and $\beta$ values. Like in regular negamax search, `ai_turn = True` depicts that the current player tries to maximize the score and `ai_turn = False` depicts that the current player tries to minimize the score. It handles alpha-beta pruning as follows:
1. If `ai_turn = True` and `score >= beta`, end the search and return the current result. Per the definition above, if the score is greater than or equal to $\beta$, we can cut the search short, saving computation time.
1. If `ai_turn = True` and `score > alpha`, set $\alpha$ to the current score. This is the minimum score that the maximizing player can achieve.
1. If `ai_turn = False` and `score <= alpha`, end the search and return the current result. Per the definition above, if the score is less than or equal to $\alpha$, we can cut the search short, saving computation time.
1. If `ai_turn = False` and `score < beta`, set $\beta$ to the current score. This is the maximum score that the minimizing player can achieve.

Optionally, this function implements memoization. This works slightly differently from memoization in standard negamax search and is detailed [later in this notebook](AlphaBetaAlgorithm.ipynb#Memoization-For-Alpha-Beta-Pruning).

Furthermore, this function implements singular value extension, which works the same as in standard negamax search.

### Dependencies

In [ ]:
import chess
import chess.gaviota
from enum import Enum

import import_ipynb
import Evaluation
import Util
from Globals import *
from ChessAlgorithm import ChessAlgorithm

### AlphaBetaPruning Class
A class that implements the negamax search algorithm with alpha-beta pruning as defined above.

In [ ]:
class AlphaBetaPruning(ChessAlgorithm):
    enable_quiescence = False # Quiescence search is disabled in this version;
                              # more details can be found at the bottom of this notebook.

#### AlphaBetaPruning.get_best_move
Finds the best move to make based on the negamax search algorithm with alpha-beta pruning.

__This function is implemented recursively, but indirectly in the following sequence:__
* `AlphaBetaPruning.get_best_move` calls `AlphaBetaPruning.evaluate_current_node` to find the best move at the current node in the search tree.
* `AlphaBetaPruning.evaluate_current_node` calls `AlphaBetaPruning.evaluate_moves` to go through all legal moves at the current node and evaluate them.
* `AlphaBetaPruning.evaluate_moves` calls `AlphaBetaPruning.get_best_move` to find the best move to make after the move it's evaluating, i.e. the best move at a child node.

###### <u>__Arguments__</u>
``alpha (int):``  
The "alpha" value. When the function is first called, this value should be strongly negative.  

``beta (int):``  
The "beta" value. When the function is first called, this value should be strongly positive.  

``ai_turn (bool):``  
Whether or not the current turn is of the AI that started the search. If this is the case, the score should be maximized. Otherwise, the score should be minimized.  
 
``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

``last_eval_score (int):``  
The score provided by the previous evaluation in the search.  

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The recommended best move to make.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

###### __<u>Side effects</u>__
- If a cache dictionary for memoization is used, new values may be added to this dictionary.

In [ ]:
def get_best_move(
    self,
    alpha: int,
    beta: int,
    ai_turn: bool,
    depth: int,
    last_eval_score: int
) -> (int, chess.Move, bool, list):

    state = self.board.get_state_repr()
    color = self.board.turn
    original_color = color if ai_turn else not color
    
    # Read cached result if available
    if self.cache is not None:                
        alpha, beta, cached_entry = self.get_cache(depth, state, alpha, beta)
        if cached_entry is not None: return cached_entry

    # Get result if the search has finished (if max depth has been reached,
    # or if the game has ended)
    result_score = self.board.get_search_result_if_finished(original_color, depth, last_eval_score)
    if result_score is not None:
        return result_score, None, False, []

    # Get result if the search has not finished
    result = self.evaluate_current_node(alpha, beta, ai_turn, depth, last_eval_score)

    # Store result in cache
    if self.cache is not None:
        self.store_cache(depth, state, alpha, beta, result)
        
    return result

AlphaBetaPruning.get_best_move = get_best_move
del get_best_move

#### AlphaBetaPruning.evaluate_current_node
Finds the best move at the current node in the alpha-beta pruning search. This assumes that the search has not ended at this node, and that the result cannot be retrieved from the cache.

__This function is implemented recursively, but indirectly in the following sequence:__
* `AlphaBetaPruning.get_best_move` calls `AlphaBetaPruning.evaluate_current_node` to find the best move at the current node in the search tree.
* `AlphaBetaPruning.evaluate_current_node` calls `AlphaBetaPruning.evaluate_moves` to go through all legal moves at the current node and evaluate them.
* `AlphaBetaPruning.evaluate_moves` calls `AlphaBetaPruning.get_best_move` to find the best move to make after the move it's evaluating, i.e. the best move at a child node.

###### __<u>Arguments</u>__
The arguments are the same as in ``AlphaBetaPruning.get_best_move``. To save space, these will not be repeated here.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The recommended best move to make.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def evaluate_current_node(
    self,
    alpha: int,
    beta: int,
    ai_turn: bool,
    depth: int,
    last_eval_score: int
) -> (int, chess.Move, bool, list):

    available_moves = self.get_move_list(ai_turn, depth)
    return self.evaluate_moves(available_moves, alpha, beta, ai_turn, depth, last_eval_score)

AlphaBetaPruning.evaluate_current_node = evaluate_current_node
del evaluate_current_node

#### AlphaBetaPruning.get_move_list
Returns a list of legal moves at the current board state. This is its own function because progressive deepening requires this list to be sorted, which is described in further detail in a [later section](AlphaBetaAlgorithm.ipynb#Progressive-deepening). For regular alpha-beta pruning, the list is not sorted.

###### __<u>Arguments</u>__
``ai_turn (bool):``  
Whether or not it's the turn of the AI that started the search.  

``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

###### __<u>Returns _(list)_</u>__
The list of legal moves.

In [ ]:
def get_move_list(self, ai_turn: bool, depth: int) -> list:
    return self.board.legal_moves

AlphaBetaPruning.get_move_list = get_move_list
del get_move_list

#### AlphaBetaPruning.evaluate_moves
Iterates through the available legal moves and searches their move trees using alpha-beta pruning. Once all moves are evaluated this way, the best move and its associated properties are returned.

Furthermore, this function implements singular value extension. If there's only one legal move to be made, the computation time of the current node is neglegible. Therefore, if this is the case, this node does not count towards the search depth.

This function also implements quiescence search, if it's enabled. More information on quiescence search can be found at the [bottom of this notebook](AlphaBetaAlgorithm.ipynb#Quiescence).

__This function is implemented recursively, but indirectly in the following sequence:__
* `AlphaBetaPruning.get_best_move` calls `AlphaBetaPruning.evaluate_current_node` to find the best move at the current node in the search tree.
* `AlphaBetaPruning.evaluate_current_node` calls `AlphaBetaPruning.evaluate_moves` to go through all legal moves at the current node and evaluate them.
* `AlphaBetaPruning.evaluate_moves` calls `AlphaBetaPruning.get_best_move` to find the best move to make after the move it's evaluating, i.e. the best move at a child node.

###### __<u>Arguments</u>__
``available_moves (list<chess.Move>):``  
A list of available legal moves.  

The other arguments are the same as in `AlphaBetaPruning.get_best_move`. To save space, these will not be repeated here.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The recommended best move to make.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def evaluate_moves(
    self,
    available_moves: list,
    alpha: int,
    beta: int,
    ai_turn: bool,
    depth: int,
    last_eval_score: int
)  -> (int, chess.Move, bool, list):

    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False
    best_move_path = []

    use_singular_value_extension = (len(list(available_moves)) == 1)
    for move in available_moves:
        eval_score, used_endgame_anywhere = self.board.evaluate_move(self.use_heuristic,
                not ai_turn, last_eval_score, depth, move, self.endgame_tablebase)

        use_quiescence = (depth == 1 and self.board.is_capture(move)) \
            if self.enable_quiescence \
            else False

        self.board.push(move)
        
        if self.board.is_game_over():
            score_after_move = eval_score
            new_move_path = []
        else:
            new_depth = depth if use_singular_value_extension or use_quiescence else depth - 1
            score_after_move, _, used_endgame, new_move_path = self.get_best_move(
                    alpha, beta, not ai_turn, new_depth, eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame
                
        self.board.pop()          
                                                   
        if (ai_turn and score_after_move > best_score) \
                or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere
            best_move_path = [move] + new_move_path

        if ai_turn:
            if best_score >= beta:
                return best_score, best_move, best_move_used_endgame, best_move_path
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, best_move, best_move_used_endgame, best_move_path
            if best_score < beta:
                beta = best_score
                
    return best_score, best_move, best_move_used_endgame, best_move_path

AlphaBetaPruning.evaluate_moves = evaluate_moves
del evaluate_moves

#### AlphaBetaPruning.make_move
Finds the best possible move according to the negamax search algorithm with alpha-beta pruning and pushes it onto the move stack.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The move that was made.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def make_move(self) -> (int, chess.Move, bool, list):
    score, move, used_endgame, move_path = self.get_best_move(
        alpha=-Globals.MAX_EVALUATION_SCORE, 
        beta=Globals.MAX_EVALUATION_SCORE, 
        ai_turn=True,
        depth=self.search_depth,
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    
    self.board.push(move)
    return score, move, used_endgame, move_path

AlphaBetaPruning.make_move = make_move
del make_move

### Memoization For Alpha-Beta Pruning
Because the same state at the same remaining search depth may yield a different score if the values for $\alpha$ and $\beta$ are different, it is required to include $\alpha$ and $\beta$ in the cache when implementing memoization. While it would be correct to simply store these values in the cache and only use the cached values if $\alpha$ and $\beta$ are equal to their cached counterparts, this is not optimal. Because chess evaluation scores can vary so greatly, it is unlikely for $\alpha$ and $\beta$ to be exactly equal to their cached values, meaning that the cache can rarely be used. As described earlier, $\alpha$ and $\beta$ are bounds for evaluation scores. This property can be used to improve the percentage of cache hits while still producing a correct score, see [Stroetmann, K. (2022)](Bibliography.ipynb#KS22).

In this memoization technique, cached values are tuples $(\texttt{flag}, (\texttt{v}, \texttt{m}, \texttt{e}))$.
* $\texttt{v} \in \mathbb{Z}$ is the cached move score.  
* $\texttt{m} \in \texttt{Moves}$ is the corresponding move.  
* $\texttt{e} \in \mathbb{B}$ denotes whether or not the endgame tablebases were used to find the move.  
* $\texttt{flag}$ can be `≤`, `=`, or `≥`.  

The cache is represented by a dictionary $\texttt{cache}$. The keys in this dictionary are tuples $(\texttt{d}, \texttt{s})$.
* $\texttt{d} \in \mathbb{Z}$ is the remaining search depth, i.e. how many half-moves the search can still look into the future.
* $\texttt{s} \in \texttt{States}$ is the board state at the current node in the search tree.

This value tuples in the cache represent the following:
* $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`=`$, (\texttt{v}, \texttt{m}, \texttt{e})) \rightarrow \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta) = \texttt{v}$.
* $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`≤`$, (\texttt{v}, \texttt{m}, \texttt{e})) \rightarrow \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta) \leq \texttt{v}$.
* $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`≥`$, (\texttt{v}, \texttt{m}, \texttt{e})) \rightarrow \texttt{bestValueAlphaBeta}(\texttt{s}, \texttt{p}, \texttt{o}, \texttt{d}, \texttt{path}, \alpha, \beta) \geq \texttt{v}$.

Similarly, we implement memoization based on these three cases:
1. $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`=`$, (\texttt{v}, \texttt{m}, \texttt{e}))$:  
    In this case, the move score is exactly equal to $\texttt{v}$. Therefore, we do not change $\alpha$ and $\beta$ and simply use the cached result.

1. $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`≤`$, (\texttt{v}, \texttt{m}, \texttt{e}))$:  
    In this case, the real move score is less than or equal to $\texttt{v}$. Here, we consider three different cases:
    * $\texttt{v} \leq \alpha$. According to the rules of alpha-beta pruning, if the real score is less than or equal to $\alpha$, we must return a value less than or equal to $\alpha$. Here, we return $\texttt{v}$, but returning $\alpha$ would also have been correct.
    * $\alpha < \texttt{v} < \beta$. Hence, the real score is less than $\beta$. Therefore, we can use $\texttt{v}$ as the new upper bound, i.e. we set $\beta$ to $\texttt{v}$.
    * $\texttt{v} \geq \beta$. This means that our current upper bound ($\beta$) is smaller than the cached upper bound ($\texttt{v}$), and therefore, the cached result is not of any help.

1. $\texttt{cache}[(\texttt{d}, \texttt{s})] = ($`≥`$, (\texttt{v}, \texttt{m}, \texttt{e}))$:  
    In this case, the real move score is greater than or equal to $\texttt{v}$. Here, we consider three different cases:
    * $\texttt{v} \geq \beta$. According to the rules of alpha-beta pruning, if the real score is greater than or equal to $\beta$, we must return a value greater than or equal to $\beta$. Here, we return $\texttt{v}$, but returning $\beta$ would also have been correct.
    * $\alpha < \texttt{v} < \beta$. Hence, the real score is greater than $\alpha$. Therefore, we can use $\texttt{v}$ as the new lower bound, i.e. we set $\alpha$ to $\texttt{v}$.
    * $\texttt{v} \leq \alpha$. This means that our current lower bound ($\alpha$) is greater than the cached lower bound ($\texttt{v}$), and therefore, the cached result is not of any help.

Of note is that our implementation does not use string literals for the $\texttt{flag}$ parameter in the cached value tuple. Instead, it uses integer values (retrieved using an enumerator) for slightly better performance.

### AlphaBetaPruningMemo Class
A class that implements the negamax search algorithm with alpha-beta pruning, including memoization as defined above.

In [ ]:
class AlphaBetaPruningMemo(AlphaBetaPruning):
    pass

### Comparisons Class
An enumerator that contains values for the following comparisons:
- Equal to
- Less than or equal to
- Greater than or equal to

These values are used for alpha-beta pruning memoization and replace the single-character strings mentioned in the theory above, as integer constants are more efficient.

In [ ]:
class Comparisons(Enum):
    EQUAL = 0
    LTE = 1 # Less than or equal to
    GTE = 2 # Greater than or equal to

#### AlphaBetaPruningMemo.get_cache
Retrieves a cached value in an alpha-beta pruning search, based on the equations listed in the theory above.

###### __<u>Arguments</u>__
``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

``state (tuple):``  
A representation of the current chess board state.  

``alpha (int):``  
The current "alpha" value.  

``beta (int):``  
The current "beta" value.

###### __<u>Returns _(int, int, (int, chess.Move, bool, list<chess.Move>))_</u>__
- The new "alpha" value.
- The new "beta" value.
- The move score, move, whether or not the endgame tablebases were used and the predicted move sequence; or `None` if no value could be read from the cache.

In [ ]:
def get_cache(
    self,
    depth: int,
    state: tuple,
    alpha: int,
    beta: int
) -> (int, int, (int, chess.Move, bool, list)):

    key = (depth, state)
    if key not in self.cache:
        return alpha, beta, None

    flag, result = self.cache[key]
    score, _, _, _ = result
    if flag == Comparisons.EQUAL:
        return alpha, beta, result
    if flag == Comparisons.LTE:
        if score <= alpha:
            return alpha, beta, result
        if score < beta: # alpha < score < beta
            return alpha, score, None
    if flag == Comparisons.GTE:
        if beta <= score:
            return alpha, beta, result
        if alpha < score: # alpha < score < beta
            return score, beta, None

    # Invalid flag or no optimization possible
    return alpha, beta, None

AlphaBetaPruningMemo.get_cache = get_cache
del get_cache

#### AlphaBetaPruningMemo.store_cache
Caches a board score in an alpha-beta pruning search, based on the representation of the cached tuples as described in the theory above.

###### __<u>Arguments</u>__
``depth (int):``  
The remaining depth of the search, i.e. how many half- moves the search should still look into the future.  

``state (tuple):``  
A representation of the current chess board state.  

``alpha (int):``  
The current "alpha" value.  

``beta (int):``  
The current "beta" value.  

``result (tuple<int, chess.Move, bool, list<chess.Move>>):``  
The move score, move, whether or not the endgame tablebases were used and the predicted move sequence.

In [ ]:
def store_cache(
    self,
    depth: int,
    state: tuple,
    alpha: int,
    beta: int,
    result: tuple
) -> None:

    key = (depth, state)
    score, _, _, _ = result

    if score <= alpha:
        self.cache[key] = (Comparisons.LTE, result)
    elif score < beta: # alpha < score < beta
        self.cache[key] = (Comparisons.EQUAL, result)
    else: # beta <= score
        self.cache[key] = (Comparisons.GTE, result)

AlphaBetaPruningMemo.store_cache = store_cache
del store_cache

#### AlphaBetaPruningMemo.make_move
Finds the best possible move according to the negamax search algorithm with alpha-beta pruning and memoization, and pushes it onto the move stack.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The move that was made.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def make_move(self) -> (int, chess.Move, bool, list):
    self.cache = {}
    score, move, used_endgame, move_path = self.get_best_move(
        alpha=-Globals.MAX_EVALUATION_SCORE, 
        beta=Globals.MAX_EVALUATION_SCORE, 
        ai_turn=True,
        depth=self.search_depth,
        last_eval_score=0 # The starting board score doesn't matter,
                          # it's evaluated by score difference
    )
    
    self.board.push(move)
    return score, move, used_endgame, move_path

AlphaBetaPruningMemo.make_move = make_move
del make_move

### Progressive deepening

__Progressive deepening__, also known as iterative deepening, is the idea of gradually increasing the maximum search depth up to a certain number, see [Chess Programming Wiki (2019)](Bibliography.ipynb#CPW19). In the context of games, this method can be used if information from a previous search is useful for a current search, even if that previous search used a lower search depth.

According to [Chess Programming Wiki (2022a)](Bibliography.ipynb#CPW22a), this is the case in alpha-beta pruning. In alpha-beta pruning, it is advantageous to evaluate the best moves first. These moves change the values of $\alpha$ and $\beta$ the most significantly. Therefore, more branches of the search can be pruned, decreasing the amount of nodes to evaluate and speeding up the search. If the move scores of previous searches in a progressive deepening implementation are cached using memoization, these values can be used to estimate the effectiveness of each move, allowing the current search to sort the move list using these scores. This creates a best-first ordered list of moves, allowing the aforementioned optimization to be implemented.

Here, it should be taken into account whether the current player tries to maximize or minimize the score. If they want to maximize the score, the list should be sorted in descending order. If they want to minimize the score, the list should be sorted in ascending order.

Because the amount of nodes that a search needs to evaluate grows exponentially with its search depth, searching the tree with a lower search depth takes comparatively little time. Generally, the time saved from sorting moves outweighs the time spent on performing these extra searches.

### AlphaBetaPruningPD Class
A class that implements the negamax search algorithm with alpha-beta pruning, memoization and progressive deepening.

In [ ]:
class AlphaBetaPruningPD(AlphaBetaPruningMemo):
    pass

#### AlphaBetaPruningPD.get_move_list
Returns a list of legal moves, sorted by move score. This lets alpha-beta pruning search through the best moves first.

###### __<u>Arguments</u>__
``ai_turn (bool):``  
Whether or not it's the turn of the AI that started the search, i.e. whether the current player is maximizing or minimizing. The list will be sorted either descendingly or ascendingly depending on this value.  

``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

###### __<u>Returns _(list)_</u>__
The sorted list of legal moves.

In [ ]:
def get_move_list(self, ai_turn: bool, depth: int) -> list:
    return sorted(
        self.board.legal_moves,
        reverse=ai_turn,
        key=lambda move: self.value_cache(move, depth - 2)
    )

AlphaBetaPruningPD.get_move_list = get_move_list
del get_move_list

#### AlphaBetaPruningPD.value_cache
Returns a cached move score if available, or a score of 0 if not. Can be used to sort moves by score.

###### __<u>Arguments</u>__
``move (chess.Move):``  
The move whose potentially cached score should be found.  

``depth (int):``  
The remaining depth of the search, i.e. how many half-moves the search should still look into the future.  

###### __<u>Returns _(int)_</u>__
The cached move score, or 0 if unavailable.

In [ ]:
def value_cache(self, move: chess.Move, depth: int) -> int:
    self.board.push(move)
    state = self.board.get_state_repr()
    self.board.pop()
    
    _, result = self.cache.get((depth, state), ('=', (0, None, False, [])))
    score, _, _, _ = result
    return score

AlphaBetaPruningPD.value_cache = value_cache
del value_cache

#### AlphaBetaPruningPD.make_move
Finds the best possible move according to the negamax search algorithm with alpha-beta pruning, memoization and progressive deepening, and pushes it onto the move stack.

###### __<u>Returns _(int, chess.Move, bool, list<chess.Move>)_</u>__
- The evaluated score of the best possible move.
- The move that was made.
- Whether or not the endgame tablebases were used to find the move.
- The predicted move path, i.e. the sequence of moves that the algorithm predicts will take place.

In [ ]:
def make_move(self) -> (int, chess.Move, bool):
    score, move, used_endgame, move_path = 0, None, False, []
    self.cache = {}

    for limit in range(1, self.search_depth + 1):
        score, move, used_endgame, move_path = self.get_best_move(
            alpha=-Globals.MAX_EVALUATION_SCORE,
            beta=Globals.MAX_EVALUATION_SCORE,
            ai_turn=True,
            depth=limit,
            last_eval_score=0 # The starting board score doesn't matter,
                              # it's evaluated by score difference
        )

        # If a checkmate in favor of the AI has been found, there's no
        # reason to search any further.
        if score >= Globals.EVALUATION_SCORE_CHECKMATE:
            self.board.push(move)

            # The score should be calculated based on the limit and the search depth within
            # that limit. The lower the limit the better, and the higher the search depth
            # the better. This score matches the corresponding score from alpha-beta
            # pruning without progressive deepening.
            score = Globals.EVALUATION_SCORE_CHECKMATE + self.search_depth - limit + 1
            return score, move, used_endgame, move_path

    self.board.push(move)
    return score, move, used_endgame, move_path

AlphaBetaPruningPD.make_move = make_move
del make_move

<a id= 'Quiescence'></a>
### Quiescence Search

One problem with the search algorithms thus far is that they have a fixed maximum search depth. Ignoring singular value extension, these algorithms are unable to look further than the maximum search depth given to them. This can be problematic in certain circumstances. As an example, consider the following simple board state where white can make the next move.

![](Images/QuiescenceExample.png)

Let us assume that the white player is a search algorithm that reached this position with a remaining search depth of 1. In other words, it is only able to look one more half-move into the future. There are several legal moves to make in this position, so singular value extension is not of relevance here.

One such legal move is `d2d4`, where the white queen captures the black knight. Capturing a piece results in a high evaluation score, so the search algorithm considers this a great move for white. However, it is clear that this is very inaccurate, as the black knight is protected by the black pawn on `E5`. As a result, if black makes the move `e5d4`, it can capture the white queen. This is not a favorable trade for white, making `d2d4` a terrible move in practice.

If the search algorithm were to look one more half-move into the future, it could find the move `e5e4` from black and realize that `d2d4` is a bad move. However, as it stands, it is unable to do so. As far as this algorithm is concerned, the move `d2d4` lets white capture a knight, making this an excellent move. It will therefore rate the path that leads to this position much higher than it should, resulting in the algorithm potentially choosing a seemingly nonsensical move in an attempt to reach this position.

In other words, these algorithms cannot recognize protected pieces if their remaining search depth is 1. A solution to this problem is **quiescence search**. The idea behind quiescence search is that a search may only end on a "quiet" position, see [Chess Programming Wiki (2021a)](Bibliography.ipynb#CPW21a). In our implementation, a quiet position is defined as a position where the last move was not a capture. Traditionally, checks are also considered non-quiet. However, the function `is_check` from the used chess library is implemented very inefficiently, therefore checking for check is a slow operation. From testing, we found that this additional check could make the search up to four times slower, so we decided to omit this check and only consider captures.

Implementing quiescense search is similar to singular value extension in that a node does not count towards the search depth. In this case, any non-quiet node that would normally be a leaf node is not counted, with the search continuing down the branches from this node. After that, once a quiet node is reached, the search of the current branch is considered finished.

Naturally, because quiescence search has to search deeper than usual in some branches, it slows down the search. As such, if quiescence search is enabled, it is recommended to use a lower search depth to compensate. This means that the search algorithm isn't able to look as far ahead for quiet positions, even if those quiet positions may lead to checkmates. However, it is also less likely to make nonsensical moves as a result of not detecting protected pieces.

### AlphaBetaPruningQuiescence Class
A class that implements the negamax search algorithm with alpha-beta pruning, memoization, progressive deepening and quiescence search.

In [ ]:
class AlphaBetaPruningQuiescence(AlphaBetaPruningPD):
    enable_quiescence = True